In [1]:
# Import libraries
import numpy as np
from mp_api.client import MPRester
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure
from pymatgen.analysis.elasticity import *
from pymatgen.io.vasp.inputs import *
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os


# Import simtool libraries
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

# Import helper functions
from qe_functions import *

In [2]:
simToolName = "elasticdft"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print("%18s = %s" % (key,simToolLocation[key]))

      notebookPath = /home/nanohub/rapplet/elastic-dft-qe/simtool/elasticdft.ipynb
       simToolName = elasticdft
   simToolRevision = None
         published = False


In [3]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

elasticdft:
   installed:
      r6: Using quantum espresso to compute the stresses associated with different
         strain states and fitting the elastic tensor.


In [4]:
# get the list of inputs
inputs = getSimToolInputs(simToolLocation)
print(inputs)

mp_id:
    type: Text
    description: Materials Project ID for chosen structure

squid_id:
    type: Text
    description: Simulation ID for relaxed structure from CellRelaxDFT

strains:
    type: List
    description: List of strain magnitudes

shears:
    type: List
    description: List of shear magnitudes



In [5]:
# get the list of outputs
outputs = getSimToolOutputs(simToolLocation)
print(outputs)

external_pressure:
    type: Number
    description: External pressure of the prestine structure queried from CellRelaxDFT
    units: gigapascal

strain_matrices:
    type: Array
    description: Array of strain matrices that were actually used in the generation of the deformed structures and fitting of the elastic tensor (only different from the input if sym_red is True)

stress_tensors:
    type: Array
    description: Array of stress tensors as arrays that were computed by quantum espresso for each deformed structure and used in the fit for the elastic tensor

deformed_structures:
    type: List
    description: List of pymatgen Structure objects in dictionary format that were generated by the applying strains to the pristine structure

energies:
    type: List
    description: List of energies per formula unit from the DFT calculations on each deformed structure

elastic_tensor:
    type: Array
    description: Elastic tensor in voigt notation fit with pymatgen using the Moore-Penr

In [6]:
mp_id = "mp-8062"
squid_id = "cellrelaxdft/r58/b7599a4fad2901a212dd6c1eca867731ed105075"

In [7]:
strains = [-0.01,-0.005,0.005,0.01]
shears = [-0.02,-0.01,0.01,0.02]

In [8]:
inputs['mp_id'].value = mp_id
inputs['squid_id'].value = squid_id
inputs['strains'].value = strains
inputs['shears'].value = shears

In [9]:
inputs

mp_id:
    type: Text
    description: Materials Project ID for chosen structure
    value: mp-8062

squid_id:
    type: Text
    description: Simulation ID for relaxed structure from CellRelaxDFT
    value: cellrelaxdft/r58/b7599a4fad2901a212dd6c1eca867731ed105075

strains:
    type: List
    description: List of strain magnitudes
    value: [-0.01, -0.005, 0.005, 0.01]

shears:
    type: List
    description: List of shear magnitudes
    value: [-0.02, -0.01, 0.01, 0.02]

In [10]:
r = Run(simToolLocation,inputs)

runname = dbf9ca53ea0e4ba194f22c0914db18f3
outdir  = RUNS/dbf9ca53ea0e4ba194f22c0914db18f3
cached  = False
submit --local
       /apps/share64/debian10/anaconda/anaconda-7/envs/MatProject/bin/papermill
       --no-request-save-on-cell-execute --autosave-cell-every 0
       --no-use-black-format-injection --parameters_file inputs.yaml
       /home/nanohub/rapplet/elastic-dft-qe/simtool/elasticdft.ipynb
       elasticdft.ipynb


Input Notebook:  /home/nanohub/rapplet/elastic-dft-qe/simtool/elasticdft.ipynb
Output Notebook: elasticdft.ipynb
Executing notebook with kernel: python3
Executing: 100%|██████████| 22/22 [48:41<00:00, 132.78s/cell]


In [11]:
r.getResultSummary()

,name,data,encoder,display,filename
0,simToolSaveErrorOccurred,0,text,None,elasticdft.ipynb
1,simToolAllOutputsSaved,1,text,None,elasticdft.ipynb
2,external_pressure,-0.145,text,None,elasticdft.ipynb
3,strain_matrices,"[[[-0.01, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0....",text,None,elasticdft.ipynb
4,stress_tensors,"[[[-3.7990000000000004, -0.0, -0.0], [-0.0, -1...",text,None,elasticdft.ipynb
5,deformed_structures,"[{""@module"": ""pymatgen.core.structure"", ""@clas...",text,None,elasticdft.ipynb
6,energies,"[-867.9545457327689, -867.95632542439, -867.95...",text,None,elasticdft.ipynb
7,elastic_tensor,"[[390.18000000000006, 136.3, 136.2600000000000...",text,None,elasticdft.ipynb
8,C11,390.18000000000006,text,None,elasticdft.ipynb
9,C12,136.36,text,None,elasticdft.ipynb
